In [17]:
import os
import pandas as pd
from sqlalchemy import create_engine, inspect
from dotenv import load_dotenv

In [13]:
load_dotenv()

USER = os.getenv("POSTGRES_USER")
PASSWORD = os.getenv("POSTGRES_PASSWORD")
PORT = os.getenv("POSTGRES_PORT")
HOST = "localhost"
DB_NAME = "db_soccerstats"
    
engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")

In [14]:
raw_path = "data/raw/"
csv_files = [f for f in os.listdir(raw_path) if f.endswith(".csv")]
print("Arquivos encontrados:\n", csv_files)

Arquivos encontrados:
 ['2015_scouts.csv', '2014_partidas.csv', '2014_scouts.csv', '2015_atletas.csv', 'posicoes.csv', '2016_scouts.csv', '2016_clubes.csv', '2014_atletas.csv', '2017_clubes.csv', '2014_clubes.csv', '2017_atletas.csv', '2017_partidas.csv', '2016_partidas.csv', '2015_partidas.csv', 'status.csv', '2016_atletas.csv', 'pontuacao.csv', '2015_clubes.csv', '2017_scouts.csv']


In [15]:
def create_table(df, table_name, schema_name):
    df.to_sql(table_name, engine, schema=schema_name, if_exists="replace", index=False)
    print(f"Tabela '{schema_name}.{table_name}' criada com {len(df)} registros!")

In [16]:
for csv_file in csv_files:
    df_raw = pd.read_csv(os.path.join(raw_path, csv_file))
    table_name = csv_file.replace(".csv", "").lower()
    create_table(df_raw, table_name, schema_name="raw")

Tabela 'raw.2015_scouts' criada com 30510 registros!
Tabela 'raw.2014_partidas' criada com 380 registros!
Tabela 'raw.2014_scouts' criada com 31567 registros!
Tabela 'raw.2015_atletas' criada com 1026 registros!
Tabela 'raw.posicoes' criada com 6 registros!
Tabela 'raw.2016_scouts' criada com 31168 registros!
Tabela 'raw.2016_clubes' criada com 20 registros!
Tabela 'raw.2014_atletas' criada com 1033 registros!
Tabela 'raw.2017_clubes' criada com 20 registros!
Tabela 'raw.2014_clubes' criada com 20 registros!
Tabela 'raw.2017_atletas' criada com 944 registros!
Tabela 'raw.2017_partidas' criada com 200 registros!
Tabela 'raw.2016_partidas' criada com 380 registros!
Tabela 'raw.2015_partidas' criada com 380 registros!
Tabela 'raw.status' criada com 5 registros!
Tabela 'raw.2016_atletas' criada com 1001 registros!
Tabela 'raw.pontuacao' criada com 18 registros!
Tabela 'raw.2015_clubes' criada com 20 registros!
Tabela 'raw.2017_scouts' criada com 16313 registros!


In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(
        text("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'raw';
        """)
    )
    raw_tables = [row[0] for row in result]

# print("Tabelas na Raw:", raw_tables)

for table_name in raw_tables:
    df = pd.read_sql(f'SELECT * FROM "raw"."{table_name}"', engine)
    
    df.columns = [col.strip().lower() for col in df.columns]
    df = df.drop_duplicates()
    # df = df.fillna("")
    
    create_table(df, table_name, schema_name="silver")


In [21]:
anos = [2014, 2015, 2016, 2017]

ordem_atributos = [
    'atleta_id', 'clube_id', 'posicao_id', 'partida_id', 'rodada', 'participou',
    'jogos_num', 'pontos_num', 'media_num', 'preco_num', 'variacao_num',
    'mando', 'titular', 'substituido', 'tempo_jogado', 'nota',
    'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS'
]

dfs = []

inspector = inspect(engine)

for ano in anos:
    table_name = f"{ano}_scouts"
    df = pd.read_sql_table(table_name, engine, schema="raw")

    for col in ordem_atributos:
        if col not in df.columns:
            df[col] = None

    df = df[ordem_atributos]
    
    dfs.append(df)

df_final = pd.concat(dfs, ignore_index=True)

df_final.to_sql('scout', engine, schema="silver", if_exists='replace', index=False)
print(f"Tabela 'silver.scout' criada com {len(df_final)} registros!")

/tmp/ipykernel_38963/3295859497.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat(dfs, ignore_index=True)


Tabela 'silver.scout' criada com 109558 registros!
